In [50]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel , RunnableBranch , RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
load_dotenv()

# -------------------------
# 1️⃣ Pydantic Schema
# -------------------------
class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(
        description="Give the sentiment of the Feedback"
    )
llm = ChatOllama(
    model="llama3.2:latest",
    temperature=1
)

In [47]:
user_input = "This is a terrible phone" 

In [ ]:
pydantic_feedback_parser = PydanticOutputParser(pydantic_object=Feedback)
prompt1 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into positive or negative:\n"
             "{feedback} , Do not output it in jason format Use only the format instructions given here \n\n{format_instructions}",
    input_variables=['feedback'],
    partial_variables={'format_instructions': pydantic_feedback_parser.get_format_instructions()}
)

chain = prompt1 | llm | pydantic_feedback_parser

classifier_output = chain.invoke({"feedback": user_input})

OutputParserException: Failed to parse Feedback from completion {"properties": {"sentiment": {"description": "Give the sentiment of the Feedback", "enum": ["positive", "negative"], "title": "Sentiment", "type": "string"}}, "required": ["sentiment"]}. Got: 1 validation error for Feedback
sentiment
  Field required [type=missing, input_value={'properties': {'sentimen...equired': ['sentiment']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 

In [54]:
prompt1

PromptTemplate(input_variables=['feedback'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sentiment": {"description": "Give the sentiment of the Feedback", "enum": ["positive", "negative"], "title": "Sentiment", "type": "string"}}, "required": ["sentiment"]}\n```'}, template='Classify the sentiment of the following feedback text into positive or negative:\n{feedback}\n\n{format_instructions}')

In [ ]:
classifier_output

Feedback(sentiment='negative')

In [ ]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback'] ) 

In [ ]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | llm | StrOutputParser()),
    (lambda x:x.sentiment == 'negative', prompt3 | llm | StrOutputParser()),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [ ]:
branch_chain.invoke(classifier_output)

'Here\'s a sample response:\n\n"Thank you for taking the time to share your thoughts with me. I apologize that my [action/product/idea] did not meet your expectations. I appreciate your honesty and would love to learn more about what I can improve on.\n\nCan you tell me more about what was disappointing? Was there anything specific that didn\'t quite work out as planned? Your feedback will help me grow and make better decisions in the future.\n\nI\'m committed to continuous improvement and value your input. If you have any suggestions or ideas, please feel free to share them with me."\n\nThis response acknowledges the negative feedback, expresses gratitude for the feedback, and asks for clarification on what specifically didn\'t meet expectations. By asking open-ended questions, you\'re creating a safe space for the person to provide more detailed information, which can help you learn from their experience and make improvements.\n\nNote: The key is to:\n\n* Acknowledge the negative sen

In [ ]:
chain = chain | branch_chain 
result = chain.invoke({'feedback': 'This is a beautiful phone'})
print(result)


OutputParserException: Failed to parse Feedback from completion {"properties": {"sentiment": {"description": "Give the sentiment of the Feedback", "enum": ["positive", "negative"], "title": "Sentiment", "type": "string"}}, "required": ["sentiment"]}. Got: 1 validation error for Feedback
sentiment
  Field required [type=missing, input_value={'properties': {'sentimen...equired': ['sentiment']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 

# manually running

In [ ]:
feedback_text = "This is a terrible phone"
str_output_parser = StrOutputParser()
# Step 2: Branch manually
if classifier_output.sentiment == "positive":
    formatted_response_prompt = prompt2.format(feedback=feedback_text)
elif classifier_output.sentiment == "negative":
    formatted_response_prompt = prompt3.format(feedback=feedback_text)
else:
    print("Could not find sentiment")
    exit()

# Step 3: Generate response
response_raw = llm.invoke(formatted_response_prompt)
final_response = str_output_parser.invoke(response_raw)

In [ ]:
print(final_response)

Here's a potential response:

"I'm sorry to hear that you're not satisfied with your phone. Can you please tell me more about what specifically isn't meeting your expectations? Is it the performance, design, or something else? I'd be happy to try and help resolve the issue or provide more information about our product."

This response acknowledges the negative feedback, shows empathy, and asks for clarification on the specific issues with the phone. By doing so, you're not taking offense at the criticism, but rather using it as an opportunity to understand the customer's concerns and potentially provide a solution.

Alternatively, if you want to add more context or information about your product, you could say:

"I appreciate your feedback, but I'd like to let you know that [insert relevant details about the phone's features or benefits]. If you have any specific questions or concerns, I'm here to help."

This response aims to provide a balanced view of the product and show that you're